In [1]:
## script is hardcoded for the milton line at the moment. See me if you want to use it for other lines
## ONLY VALID FOR TRAIN AND SUBWAY MIDDAY ASSIGNMENT. EVERY OTHER MODE HAS NOT BEEN CONVERTED.

OUTPUT_FOLDER = r""
CACHE_FOLDER = r""

MD_SCENARIO_NUMBER = 21

In [2]:
import numpy as np
import pandas as pd
#from matrix_butler import MatrixButler
import inro.modeller as m
mm = m.Modeller()
gghm_api = mm.module('gghm.internal.api_interface')
from gghm_api import MatrixButler
from gghm_api.common.pandas_utils import fast_stack, fast_unstack
from gghm_api.common.matrix_converters.matrix_converters.emme import from_mdf, to_mdf
from gghm_api.common.emme_api import frame_to_matrix, matrix_to_frame
import os
from os import path
from copy import deepcopy as dcopy
from IPython.display import display
import csv
idx = pd.IndexSlice
gghm_api.remove_jupyter_handler()
from gghm_api.tools.model_scenario import GGHMScenario
from gghm_api.common.model_logging import getLogger
from gghm_api.tools.standalone_transit_assignment import run_standalone_transit_assignment
export_network_package = mm.tool('tmg.input_output.export_network_package')
logger = getLogger("gghm.MiddayAssignment")

In [3]:
assignment_period_splits= {
      "traffic": {
        "HBE": {
          "am_inbound": 0.35,
          "am_outbound": 0.12,
          "dn_inbound": 0.027885792,
          "dn_outbound": 0.022171795,
          "ev_inbound": 0.154209006,
          "ev_outbound": 0.272584509,
          "md_inbound": 0.28534298,
          "md_outbound": 0.237805919,
          "pm_inbound": 0.214,
          "pm_outbound": 0.316
        },
        "HBM": {
          "am_inbound": 0.04,
          "am_outbound": 0.01,
          "dn_inbound": 0.001607338,
          "dn_outbound": 0.002281755,
          "ev_inbound": 0.082050859,
          "ev_outbound": 0.193412603,
          "md_inbound": 0.38067509,
          "md_outbound": 0.339972356,
          "pm_inbound": 0.36,
          "pm_outbound": 0.59
        },
        "HBO": {
          "am_inbound": 0.23,
          "am_outbound": 0.02,
          "dn_inbound": 0.019442495,
          "dn_outbound": 0.024341101,
          "ev_inbound": 0.118812454,
          "ev_outbound": 0.329140527,
          "md_inbound": 0.299997967,
          "md_outbound": 0.208265455,
          "pm_inbound": 0.34,
          "pm_outbound": 0.41
        },
        "HBS": {
          "am_inbound": 0.59,
          "am_outbound": 0.01,
          "dn_inbound": 0.005847301,
          "dn_outbound": 0.001987008,
          "ev_inbound": 0.014549482,
          "ev_outbound": 0.101483885,
          "md_inbound": 0.331678644,
          "md_outbound": 0.54445368,
          "pm_inbound": 0.05,
          "pm_outbound": 0.35
        },
        "HBU": {
          "am_inbound": 0.46,
          "am_outbound": 0.002,
          "dn_inbound": 0.009709006,
          "dn_outbound": 0.010585249,
          "ev_inbound": 0.009543755,
          "ev_outbound": 0.252455215,
          "md_inbound": 0.466465127,
          "md_outbound": 0.251241648,
          "pm_inbound": 0.09,
          "pm_outbound": 0.45
        },
        "HBW": {
          "am_inbound": 0.47,
          "am_outbound": 0.01,
          "dn_inbound": 0.156442167,
          "dn_outbound": 0.055045293,
          "ev_inbound": 0.038439266,
          "ev_outbound": 0.292337489,
          "md_inbound": 0.326070473,
          "md_outbound": 0.131665312,
          "pm_inbound": 0.042,
          "pm_outbound": 0.478
        },
        "NHB": {
          "am_inbound": 0.23,
          "am_outbound": 0,
          "dn_inbound": 0.009961403,
          "dn_outbound": 0,
          "ev_inbound": 0.221427878,
          "ev_outbound": 0,
          "md_inbound": 0.768610719,
          "md_outbound": 0,
          "pm_inbound": 0.77,
          "pm_outbound": 0
        },
        "WBO": {
          "am_inbound": 0.02,
          "am_outbound": 0.35,
          "dn_inbound": 0.011986294,
          "dn_outbound": 0.024075564,
          "ev_inbound": 0.096647922,
          "ev_outbound": 0.019615227,
          "md_inbound": 0.444555543,
          "md_outbound": 0.40311945,
          "pm_inbound": 0.56,
          "pm_outbound": 0.07
        },
        "hourly_factors": {
          "am_peak": 0.415,
          "dn_offpeak": 0.167,
          "ev_offpeak": 0.2,
          "md_offpeak": 0.2,
          "pm_peak": 0.29
        },
        "hov3_occupancy": 3.3778,
        "nonresident_business": {
          "am_inbound": 0.06555825285108016,
          "am_outbound": 0.06555825285108016,
          "dn_inbound": 0.060835851132573046,
          "dn_outbound": 0.060835851132573046,
          "ev_inbound": 0.05411201732478737,
          "ev_outbound": 0.05411201732478737,
          "md_inbound": 0.19982186206535216,
          "md_outbound": 0.19982186206535216,
          "pm_inbound": 0.1196720166262072,
          "pm_outbound": 0.1196720166262072
        },
        "nonresident_leisure": {
          "am_inbound": 0.04854924322139491,
          "am_outbound": 0.04854924322139491,
          "dn_inbound": 0.021386386336229503,
          "dn_outbound": 0.021386386336229503,
          "ev_inbound": 0.11009450128774288,
          "ev_outbound": 0.11009450128774288,
          "md_inbound": 0.1487125911524881,
          "md_outbound": 0.1487125911524881,
          "pm_inbound": 0.1712572780021446,
          "pm_outbound": 0.1712572780021446
        },
        "resident_business": {
          "am_inbound": 0.09804797084584924,
          "am_outbound": 0.09804797084584924,
          "dn_inbound": 0.051597073760362266,
          "dn_outbound": 0.051597073760362266,
          "ev_inbound": 0.06179200606175412,
          "ev_outbound": 0.06179200606175412,
          "md_inbound": 0.17586122912889346,
          "md_outbound": 0.17586122912889346,
          "pm_inbound": 0.11270172020314091,
          "pm_outbound": 0.11270172020314091
        },
        "resident_leisure": {
          "am_inbound": 0.07327625783661741,
          "am_outbound": 0.07327625783661741,
          "dn_inbound": 0.0306079327412492,
          "dn_outbound": 0.0306079327412492,
          "ev_inbound": 0.07716541001341914,
          "ev_outbound": 0.07716541001341914,
          "md_inbound": 0.1726878806387574,
          "md_outbound": 0.1726878806387574,
          "pm_inbound": 0.1462625187699569,
          "pm_outbound": 0.1462625187699569
        },
        "special_events": {
          "am_inbound": 0,
          "am_outbound": 0,
          "dn_inbound": 0,
          "dn_outbound": 0,
          "ev_inbound": 0.25,
          "ev_outbound": 0.25,
          "md_inbound": 0.125,
          "md_outbound": 0.125,
          "pm_inbound": 0.125,
          "pm_outbound": 0.125
        },
        "visitors": {
          "am_inbound": 0,
          "am_outbound": 0,
          "dn_inbound": 0,
          "dn_outbound": 0,
          "ev_inbound": 0.25,
          "ev_outbound": 0.25,
          "md_inbound": 0.125,
          "md_outbound": 0.125,
          "pm_inbound": 0.125,
          "pm_outbound": 0.125
        }
      },
      "transit": {
            "AM2hr": {
                "HBM": {
                  "outbound": 0.0032692167989436635,
                  "inbound": 0.04730310566471011
                },
                "HBO": {
                  "outbound": 0.008516508271267167,
                  "inbound": 0.24323687952038608
                },
                "HBS": {
                  "outbound": 0.0011495418190018104,
                  "inbound": 0.5030994795871051
                },
                "HBU": {
                  "outbound": 0.0003315524993467738,
                  "inbound": 0.41427761094815313
                },
                "HBW": {
                  "outbound": 0.008038224887969205,
                  "inbound": 0.3859405779112147
                },
                "WBO": {
                  "outbound": 0.2322584126180037,
                  "inbound": 0.010078042228358803
                },
                "NHB": {
                  "outbound": 0.0,
                  "inbound": 0.1318263
                }
            },
            "MD": {
                "HBM": {
                  "outbound": 0.300351341626158,
                  "inbound": 0.483489867423924
                },
                "HBO": {
                  "outbound": 0.245070930682851,
                  "inbound": 0.399957337615813
                },
                "HBS": {
                  "outbound": 0.615059297766477,
                  "inbound": 0.277634649765815
                },
                "HBU": {
                  "outbound": 0.217134875095814,
                  "inbound": 0.510298108468708
                },
                "HBW": {
                  "outbound": 0.083330863406323,
                  "inbound": 0.372763530813323
                },
                "WBO": {
                  "outbound": 0.463320072454861,
                  "inbound": 0.379936485326972 
                },
                "NHB": {
                  "outbound": 0.0,
                  "inbound": 0.847375946592012
                }
            },
            "PM2hr": {
                "HBM": {
                  "outbound":0.3573500352028360 ,
                  "inbound": 0.1666558815348640
                },
                "HBO": {
                  "outbound":0.2563762479017650 ,
                  "inbound": 0.1154324831939610
                },
                "HBS": {
                  "outbound":0.4125852956334990 ,
                  "inbound": 0.0038024934506015
                },
                "HBU": {
                  "outbound":0.2772677751402600 ,
                  "inbound": 0.0217309803850367
                },
                "HBW": {
                  "outbound": 0.3383398700595550,
                  "inbound": 0.0129787984818270
                },
                "WBO": {
                  "outbound": 0.0507366750750711,
                  "inbound":  0.4507880190827500
                },
                "NHB": {
                  "outbound": 0.0,
                  "inbound": 0.5873544441163680
                }
            },
            "EV": {
                "HBM": {
                  "outbound": 0.178030021514287,
                  "inbound": 0.032032581327241
                },
                "HBO": {
                  "outbound": 0.255133531723879,
                  "inbound": 0.056025537050941
                },
                "HBS": {
                  "outbound": 0.100880865739540 ,
                  "inbound": 0.000842689962071
                },
                "HBU": {
                  "outbound": 0.255899098737557,
                  "inbound": 0.003172834529105
                },
                "HBW": {
                  "outbound": 0.341066542637000,
                  "inbound": 0.031184194081377
                },
                "WBO": {
                  "outbound": 0.013146074150908,
                  "inbound":  0.133895777537723
                },
                "NHB": {
                  "outbound": 0.0,
                  "inbound": 0.143906232944105
                }
            },
            "AM3hr": {
                "HBW": {
                  "outbound": 0.009728946 ,
                  "inbound": 0.486415117
                },
                "HBU": {
                  "outbound": 0.000331552 ,
                  "inbound": 0.464798755
                },
                "HBO": {
                  "outbound": 0.009340952 ,
                  "inbound": 0.292923089
                },
                "HBM": {
                  "outbound": 0.003269217 ,
                  "inbound": 0.049159624
                },
                "HBS": {
                  "outbound": 0.001149542 ,
                  "inbound": 0.519163055
                },
                "WBO": {
                  "outbound": 0.258777741 ,
                  "inbound": 0.010969149
                },
                "NHB": {
                  "outbound": 0.0 ,
                  "inbound": 0.151968844
                }
            },
            "PM4hr": {
                "HBW": {
                  "outbound": 0.479646735 ,
                  "inbound": 0.024209202
                },
                "HBU": {
                  "outbound": 0.494551673 ,
                  "inbound": 0.04031802
                },
                "HBO": {
                  "outbound": 0.45242822 ,
                  "inbound": 0.245307739
                },
                "HBM": {
                  "outbound": 0.653868968 ,
                  "inbound": 0.293702192
                },
                "HBS": {
                  "outbound": 0.472101443 ,
                  "inbound": 0.00758596
                },
                "WBO": {
                  "outbound": 0.107220402 ,
                  "inbound": 0.623032708
                },
                "NHB": {
                  "outbound": 0. ,
                  "inbound": 0.848031156
                }
            },
            "ON": {
                "HBW": {
                  "outbound": 0.034923017 ,
                  "inbound": 0.136731852
                },
                "HBU": {
                  "outbound": 0.003454322 ,
                  "inbound": 0.010040761
                },
                "HBO": {
                  "outbound": 0.028285299 ,
                  "inbound": 0.015527364
                },
                "HBM": {
                  "outbound": 0.00423884 ,
                  "inbound": 0.001857348
                },
                "HBS": {
                  "outbound": 0.001972097 ,
                  "inbound": 0.0036104
                },
                "WBO": {
                  "outbound": 0.005347301 ,
                  "inbound": 0.00435429
                },
                "NHB": {
                  "outbound": 0.0 ,
                  "inbound": 0.00871782
                }
            }
        }
    }

airport_factors = {
    "resident_leisure": {
        "am_inbound": 0.00332,
        "am_outbound": 0.00332,
        "md_inbound": 0.11303,
        "md_outbound": 0.11303,
        "pm_inbound": 0.09375,
        "pm_outbound": 0.09375,
        "ev_inbound": 0.23404,
        "ev_outbound": 0.23404,
        "dn_inbound": 0.05585,
        "dn_outbound": 0.05585
    },
    "resident_business":{
        "am_inbound": 0,
        "am_outbound": 0,
        "md_inbound": 0.07454,
        "md_outbound": 0.07454,
        "pm_inbound": 0.19674,
        "pm_outbound": 0.19674,
        "ev_inbound": 0.19674,
        "ev_outbound": 0.19674,
        "dn_inbound": 0.03197,
        "dn_outbound": 0.03197
    },
    "nonresident_leisure": {
        "am_inbound": 0.00203,
        "am_outbound": 0.00203,
        "md_inbound": 0.0972,
        "md_outbound": 0.0972,
        "pm_inbound": 0.1496,
        "pm_outbound": 0.1496,
        "ev_inbound": 0.19229,
        "ev_outbound": 0.19229,
        "dn_inbound": 0.05887,
        "dn_outbound": 0.05887
    },
    "nonresident_business":{
        "am_inbound": 0,
        "am_outbound": 0,
        "md_inbound": 0.09357,
        "md_outbound": 0.09357,
        "pm_inbound": 0.10153,
        "pm_outbound": 0.10153,
        "ev_inbound": 0.2032,
        "ev_outbound": 0.2032,
        "dn_inbound": 0.10171,
        "dn_outbound": 0.10171
    }
}

In [4]:
build = GGHMScenario(
    CACHE_FOLDER,
    OUTPUT_FOLDER)

In [5]:
purposes = ['hbw','hbu','hbo','hbm','hbs','wbo','nhb']

In [6]:
for i, purpose in enumerate(purposes):
    train_purp_offpeak_s2s_pa_df = build.butler.load_matrix('trips.offpeak.%s.transit.train.s2s'%purpose)
    subway_purp_offpeak_s2s_pa_df = build.butler.load_matrix('trips.offpeak.%s.transit.subway.s2s'%purpose)
    if i == 0:
        train_md_s2s_od_df = (train_purp_offpeak_s2s_pa_df*assignment_period_splits['transit']["MD"][purpose.upper()]['inbound']+
                              train_purp_offpeak_s2s_pa_df.transpose()*assignment_period_splits['transit']["MD"][purpose.upper()]['outbound'])
        subway_md_s2s_od_df = (subway_purp_offpeak_s2s_pa_df*assignment_period_splits['transit']["MD"][purpose.upper()]['inbound']+
                              subway_purp_offpeak_s2s_pa_df.transpose()*assignment_period_splits['transit']["MD"][purpose.upper()]['outbound'])
    else:
        train_md_s2s_od_df += (train_purp_offpeak_s2s_pa_df*assignment_period_splits['transit']["MD"][purpose.upper()]['inbound']+
                              train_purp_offpeak_s2s_pa_df.transpose()*assignment_period_splits['transit']["MD"][purpose.upper()]['outbound'])
        subway_md_s2s_od_df += (subway_purp_offpeak_s2s_pa_df*assignment_period_splits['transit']["MD"][purpose.upper()]['inbound']+
                              subway_purp_offpeak_s2s_pa_df.transpose()*assignment_period_splits['transit']["MD"][purpose.upper()]['outbound'])
    

In [7]:
apm_segments = ["resident_leisure","resident_business","nonresident_leisure","nonresident_business"]

In [8]:
for segment in apm_segments:
    train_purp_offpeak_s2s_pa_df = build.butler.load_matrix('trips.offpeak.%s.transit.train.s2s'%segment)
    subway_purp_offpeak_s2s_pa_df = build.butler.load_matrix('trips.offpeak.%s.transit.subway.s2s'%segment)
    
    train_md_s2s_od_df += (train_purp_offpeak_s2s_pa_df*airport_factors[segment]['md_inbound']+
                              train_purp_offpeak_s2s_pa_df.transpose()*airport_factors[segment]['md_outbound'])
    subway_md_s2s_od_df += (subway_purp_offpeak_s2s_pa_df*airport_factors[segment]['md_inbound']+
                              subway_purp_offpeak_s2s_pa_df.transpose()*airport_factors[segment]['md_outbound'])

In [9]:
outputs_run_config_fp = os.path.join(OUTPUT_FOLDER, "run_config.json")
to_mdf(train_md_s2s_od_df, os.path.join(OUTPUT_FOLDER, "Trip Matrices", "trips_md_transit_train.mdf"))
to_mdf(subway_md_s2s_od_df, os.path.join(OUTPUT_FOLDER, "Trip Matrices", "trips_md_transit_subway.mdf"))
demand_matrices = {'train_s2s': os.path.join(OUTPUT_FOLDER, "Trip Matrices", "trips_md_transit_train.mdf"),
                   'subway_s2s':os.path.join(OUTPUT_FOLDER, "Trip Matrices", "trips_md_transit_subway.mdf"),
                   'express_bus':os.path.join(OUTPUT_FOLDER, "Trip Matrices", "trips_offpeak_transit_express_bus.mdf"),
                   'rapid_bus':os.path.join(OUTPUT_FOLDER, "Trip Matrices", "trips_offpeak_transit_rapid_bus.mdf"),
                   'bus_streetcar':os.path.join(OUTPUT_FOLDER, "Trip Matrices", "trips_offpeak_transit_local_bus.mdf"),
                   'train_subway_zts':os.path.join(OUTPUT_FOLDER, "Trip Matrices", "trips_offpeak_transit_ztsa.mdf")}

In [10]:
run_standalone_transit_assignment(run_config= outputs_run_config_fp, scenario_id = MD_SCENARIO_NUMBER, 
                                  ccr=False, demand_source="EXTERNAL", demand_items=demand_matrices,
                                  other_cache_subfolder=None, n_threads=None, period_length=None,
                                  n_iterations=1)

2024-06-06 18:41:26,403 INFO gghm.StandaloneTransitAssignment -> Validating Standalone Transit Assignment Inputs
2024-06-06 18:41:26,696 INFO gghm.StandaloneTransitAssignment -> Initializing extra transit attributes
2024-06-06 18:41:26,757 INFO gghm.StandaloneTransitAssignment -> Starting the Standalone Transit Assignment
2024-06-06 18:41:26,759 INFO gghm.TransitAssignment -> Beginning transit assignment in scenario 21
2024-06-06 18:41:27,631 INFO gghm.FunctionManager -> Setting model expressions
2024-06-06 18:41:28,821 INFO gghm.TransitAssignment -> Loading demand from external matrices
2024-06-06 18:42:02,677 INFO gghm.TransitAssignment -> Iteration 1 of 1
2024-06-06 18:45:06,453 INFO gghm.FunctionManager -> Resetting Emmebank expressions
2024-06-06 18:45:07,528 INFO gghm.StandaloneTransitAssignment -> Finshed Standalone Transit Assignment!


In [11]:
output_nwp_fp = os.path.join(OUTPUT_FOLDER, "Assigned Networks","Assigned Midday OD S2S Network.nwp")
export_network_package(scenario_number = MD_SCENARIO_NUMBER, ExportFile = output_nwp_fp, export_attributes = "ALL")
